In [0]:
# import libraries
import numpy as np
import pandas as pd
import random
from pprint import pprint

In [0]:
# load data
df = pd.read_csv("/content/drive/My Drive/PRSA_data_2010.1.1-2014.12.31.csv")

In [0]:
df = df.drop("No", axis=1)
df = df.rename(columns={"month": "label"})
l=list(df['label'])

In [0]:
# make dataset as that ,where last column contain labels
# which is  month.
# empty values filled with mean value of that column

In [0]:
df = df.drop("label", axis=1)

In [0]:
df['label']=l

In [0]:
df['pm2.5'] = df['pm2.5'].fillna((df['pm2.5'].mean()))

In [0]:
df.head()

year  day  hour      pm2.5  DEWP  TEMP    PRES cbwd    Iws  Is  Ir  label
0  2010    1     0  98.613215   -21 -11.0  1021.0   NW   1.79   0   0      1
1  2010    1     1  98.613215   -21 -12.0  1020.0   NW   4.92   0   0      1
2  2010    1     2  98.613215   -21 -11.0  1019.0   NW   6.71   0   0      1
3  2010    1     3  98.613215   -21 -14.0  1019.0   NW   9.84   0   0      1
4  2010    1     4  98.613215   -20 -12.0  1018.0   NW  12.97   0   0      1

In [0]:
# splitting teset and train data from whole dataset
def dataset_splitting(df):
    # train test data;
    train_df = pd.concat([df[df.year==2010],df[df.year== 2012 ]])
    test_df=pd.concat([df[df.year==2011],df[df.year== 2013 ],df[df.year==2014]])
    
    return train_df,test_df

In [0]:
# classify the nodes on the basis of maximum count
def node_classification(data):
    a=list( np.unique(data[:, -1]))
    b=list(data[:, -1])
    count=[]
    for i in a:
      count.append(b.count(i))
    max_=max(count)
    i=count.index(max_)

    
    return a[i]

In [0]:
# dictionary formation where less number of features are taken as a sub dimension from dataset.
def dictionary_of_splits(data, subdimension):
    
    dict_ = {}
    row,col = data.shape
    x = list(range(col- 1)) 

    if subdimension and subdimension <= len(x):
        x = random.sample(population=x, k=subdimension)
    
    # dict having all unique points for that dataset where duplicate column can also occure
    for i in x:                  
        dict_[i] = np.unique( data[:, i])
    
    return dict_

In [0]:
# here we can split the column into 2 parts upper or lower on the basis of given value
# and a particular column.
def break_node(d, col_, val_):
    if  ATTRIBUTE_LIST[col_] == "continuous":
        lower_ = d[d[:, col_] <= val_]
        upper_ = d[d[:, col_]>  val_]  
    else:
        lower_ = d[d[:, col_] == val_]
        upper_ = d[d[:, col_] != val_]
    
    return lower_, upper_


In [0]:
# this is calculation of entropy function where entropy is used for checking purity of
# particular node.
def parametric_calculation(d): 
    uniq=list(np.unique(d[:, -1]))
    all_=list(d[:, -1])
    count=[]
    for i in uniq:
      count.append(all_.count(i))
    prob=[]
    for i in  count:
      prob.append(i/sum(count))
    s=0
    for i in prob:
      s=s+(i*-np.log2(i))    
    return s

In [0]:
# here weighted sum of splitted nodes of tree are taken out
# less weighted sum tree will be perfect one
def parameters_for_purity(data_below, data_above):
 
    return ((( len(data_below) / (len(data_below) + len(data_above))) * parametric_calculation(data_below) 
                     + (len(data_above) /  (len(data_below) + len(data_above))) * parametric_calculation(data_above)))
    

In [0]:
# here weighted node tree will give perfectly split
# column and value of split for tree and then split the tree.
def find_splits(d, dict_):
    
    prev_e = 10000
    for i in dict_:
        for v in dict_[i]:
            lower_, upper_ = break_node(d, i,v)
            present_e =parameters_for_purity(lower_, upper_)
            if present_e <= prev_e:
                prev_e = present_e
                col_ = i
                val_ = v
    
    return col_, val_

In [0]:
# this is for checking catagorical condition
# if unique values are less then conscider it as catagorical if it is high then it is continuous
# and form a list of catagories.
def catagory_or_not(df):
    
    catagory = []
    for i in df.columns:
        if i != "label":
            a=False
            if type(( df[i].unique())[0])== str:
              a=True
            # here threshold is set for being catagorical is 12 bcz there are 12 classes.
            if a or (len(df[i].unique()) <= 12):
                catagory.append("categorical")
            else:
                catagory.append("continuous")
    
    return catagory

In [0]:
# algorithm for main tree formation 
# individual tree with random dimensions
def Algorithm(df, c=0, subdimension=None):

    if c != 0:
        data = df 
    else:
        global INDEX, ATTRIBUTE_LIST
        INDEX = df.columns
        ATTRIBUTE_LIST = catagory_or_not(df)
        data = df.values
                  
    
    
    # base cases
    a=a= True  if len(np.unique( data[:, -1])) == 1 else False
    b=(len(data) < min_samples)
    f= (c == max_depth)
    if a or b or f:
        return node_classification(data)

    
    # recursive part
    else:    
        c=c+1
         
        # findind the split points, column whuich is better for split and then break the
        # trees into parts upper and lower and then node form
        col_, val_ = find_splits(data,  dictionary_of_splits(data, subdimension))
        lower_, upper_ = break_node(data, col_, val_)
        
        x=len(lower_)==0
        y=len(upper_)==0
        if x or y:
            return node_classification(data)
        
        # here question for split is arises.
        question = "{} <= {}".format(INDEX[col_], val_) if ATTRIBUTE_LIST[col_] == "continuous" else  "{} = {}".format( INDEX[col_], val_)
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        # here continuation oif tree formation by right and left split.
        correct = Algorithm(lower_, c, subdimension)
        incorrect = Algorithm(upper_,c, subdimension)

        # if question is correct append it as a subtree else 
        # formation of subtree is continuous
        if correct==incorrect:
            sub_tree =correct
        else:
            sub_tree[question].append(correct)
            sub_tree[question].append(incorrect)
        
        return sub_tree

In [0]:
# calculation of accuracy between predicted and actual labels
def calculate_accuracy(predictions, labels):
    predictions_correct = predictions == labels
    return predictions_correct.mean()

In [0]:
# prediction of samples one by one
def test_for_one(example, tree):
    question = list(tree.keys())[0]
    # split question by space so that we get operator and values for prediction.
    a, opr, c = question.split(" ")
    result=(tree[question][0]  if example[a] <= float(c) else tree[question][1]) if opr == "<=" else (tree[question][0]  if str(example[a]) == c else tree[question][1])
    # base case
    x=False
    if type(result)==dict:
      x=True
    if not x:
        return result
    
    # recursive part
    else:
        return test_for_one(example, result)



In [0]:
    
# one by one predict all the labels 
def predict_All(test_df, tree):
    return test_df.apply(test_for_one, args=(tree,), axis=1)

In [0]:
# test train split
train_df, test_df = dataset_splitting(df)

In [0]:
# forest formation
def Forest_making(df, trees_, n_bootstrap, n, d, option_algorithm):
    forest = []
    for i in range(trees_):
        global max_depth ,min_samples
        max_depth=d
        min_samples=2
        if option_algorithm=='random forest':
            forest.append(Algorithm(df.iloc[np.random.randint(low=0, high=len(df), size=n_bootstrap)], subdimension=n))
        else:
            forest.append(Algorithm(df.iloc[np.random.randint(low=0, high=len(df), size=n_bootstrap)], subdimension=None))
    return forest

# algorithm foe predicting values of test data in forest and return the maximum vote of decisions.
def Forest_prediction(test_df, forest):
    dict_ = {}
    for i in range(len(forest)):
        dict_["tree_{}".format(i)] = predict_All(test_df, tree=forest[i])
    
    dict_ = pd.DataFrame(dict_)
    return dict_.mode(axis=1)[0]
    

RANDOM FOREST:

In [0]:
import math
# depth d
#  number of feature root n
# bootstraping of 800 sample formation of root n attributes
# number of trees are 8
random_forest = Forest_making(train_df,10, n_bootstrap=800,n= int(math.sqrt(len(train_df.columns))), d=20,option_algorithm='random forest') 
# prediction of test data
predictions = Forest_prediction(test_df, random_forest)
# accuracy calculation
accuracy = calculate_accuracy(predictions, test_df.label)
print("accuracy:",accuracy)

accuracy: 0.43097412480974123


In [0]:
import math
# depth d
#  number of feature root n
# bootstraping of 800 sample formation of root n attributes
# number of trees are 8
random_forest = Forest_making(train_df,10, n_bootstrap=800,n= int(len(train_df.columns)/2), d=20, option_algorithm='random forest') 
# prediction of test data
predictions = Forest_prediction(test_df, random_forest)
# accuracy calculation
accuracy = calculate_accuracy(predictions, test_df.label)
print("accuracy:",accuracy)

accuracy: 0.4472983257229833


In [0]:
import math
# depth d
#  number of feature root n
# bootstraping of 800 sample formation of root n attributes
# number of trees are 8
random_forest = Forest_making(train_df,10, n_bootstrap=800,n=len(train_df.columns), d=20,option_algorithm='random forest')



In [0]:
predictions = Forest_prediction(test_df, random_forest)
# accuracy calculation
accuracy = calculate_accuracy(predictions, test_df.label)
print("accuracy:",accuracy)

accuracy: 0.4514459665144597


BAGGING:

In [0]:

# depth d
#  number of feature root n
# bootstraping of 800 sample formation of root n attributes
# number of trees are 8

bagging = Forest_making(train_df, trees_=8, n_bootstrap=800, n=len(train_df.columns), d=20,option_algorithm='bagging')
# prediction of test data
predictions = Forest_prediction(test_df, bagging)
accuracy = calculate_accuracy(predictions, test_df.label)
# accuracy calculation
print("accuracy:",accuracy)

accuracy: 0.4332572298325723
